In [ ]:
#Notebook by Jeff Prosise (https://github.com/jeffprosise/Machine-Learning)
#Adjusted for the master HCAI - Hogeschool Utrecht by Rianne van Os
# Datasets can be found here: https://github.com/jeffprosise/Machine-Learning/tree/master/Data

# Principal Component Analysis 

[Principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA) is a mechanism for reducing the number of dimensions in data. It enjoys a number of practical applications in machine learning, including obfuscating sensitive or proprietary data, eliminating multicollinearity, removing noise from datasets, and visualizing high-dimensional data by reducing it to two or three dimensions that can be plotted. PCA doesn't directly make machine-learning models more accurate, but it can help them train faster and generalize better. And PCA is easy to perform using Scikit's [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class. Let's use it to understand how PCA works and why it can be beneficial in machine learning.

![PCA](Images/pca.png)

## Understanding PCA

To understand what PCA is and how it works, begin by using it to reduce a 2-dimensional dataset to one dimension. The first step is to generate a dataset consisting of *x* and *y* coordinate pairs that roughly form a line.

In [ ]:
import numpy as np
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
%matplotlib inline

x, y = make_regression(n_samples=200, n_features=1, noise=50, random_state=0)
x = np.interp(x, (x.min(), x.max()), (-3, 3))
y = np.interp(y, (y.min(), y.max()), (-2, 2))
xy = np.column_stack((x, y))

plt.figure(figsize=(9, 6))
plt.scatter(x, y, s=100, alpha=0.5)

Use PCA to find two principal components in the dataset.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(xy)

Now plot the two principal components. The primary component is the axis that contains most of the information. The secondary component is the axis that contains the remaining information. Because the data points roughly form a line, the primary component lies along that line.

In [ ]:
def draw_vector(v0, v1):
    ax = plt.gca()
    props = dict(arrowstyle='->', linewidth=4, mutation_scale=25)
    ax.annotate('', v1, v0, arrowprops=props)
    
plt.figure(figsize=(9, 6))
plt.scatter(x, y, s=100, alpha=0.5)

for len, vector in zip(pca.explained_variance_, pca.components_):
    draw_vector(pca.mean_, pca.mean_ + (vector * 2.5 * np.sqrt(len)))

Use the `PCA` class's `explained_variance_ratio_` attribute to quantify the amount of information contained in each component.

In [ ]:
pca.explained_variance_ratio_

More than 95% of the information in the dataset is contained in one axis (the primary component). Use PCA to reduce the dataset to one principal component.

In [ ]:
pca = PCA(n_components=1)
pca_data = pca.fit_transform(xy)

Now project all the data points onto the principal component axis by inverting the PCA transform, restoring the transformed data to two dimensions with information from just the first dimension. The original points are shown in blue, while the projected points are shown in orange.

In [ ]:
unpca_data = pca.inverse_transform(pca_data)

plt.figure(figsize=(9, 6))
plt.scatter(x, y, s=100, alpha=0.5)
plt.scatter(unpca_data[:, 0], unpca_data[:, 1], s=100, alpha=0.5)

You can see that the orange points retain most of the information contained in the blue points, even though the size of the dataset has been cut in half (from two dimensions to one). That's the gist of PCA: reducing the number of dimensions without incurring a commensurate loss of information.

## PCA with facial images

An interesting way to visualize how PCA can reduce the dimensions of a dataset while retaining most of the information is to apply it to the Labeled Faces in the Wild (LFW) dataset. We'll first load the dataset and reduce the number of dimensions by 95%. Then we'll invert the PCA transform to visualize how little of the original information was lost.

In [ ]:
from sklearn.datasets import fetch_lfw_people

faces = fetch_lfw_people(min_faces_per_person=100)
print(faces.target_names)
print(faces.images.shape)

Plot the first 24 images in the dataset along with their labels.

In [ ]:
fig, ax = plt.subplots(3, 8, figsize=(18, 10))

for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i], cmap='gist_gray')
    axi.set(xticks=[], yticks=[], xlabel=faces.target_names[faces.target[i]])

Now reduce 2,914 dimensions (62 x 47) to 150 using PCA and invert the transform to restore the facial images.

In [ ]:
pca = PCA(n_components=150, random_state=0)
pca_faces = pca.fit_transform(faces.data)
unpca_faces = pca.inverse_transform(pca_faces).reshape(1140, 62, 47)

Show the restored facial images.

In [ ]:
fig, ax = plt.subplots(3, 8, figsize=(18, 10))

for i, axi in enumerate(ax.flat):
    axi.imshow(unpca_faces[i], cmap='gist_gray')
    axi.set(xticks=[], yticks=[], xlabel=faces.target_names[faces.target[i]])

Compare the before and after images. Does it look as if 95% of the information was discarded? In fact, you can compute how much of the original information was retained by summing the explained variances. In this case, it's almost 95%:

In [ ]:
np.sum(pca.explained_variance_ratio_)

A logical question to ask is what is the "right" number of components? In other words, what number of components strikes the best balance between reducing the number of dimensions in the dataset and retaining most of the information? One way to answer that question is a scree plot, which plots the proportion of explained variance for each dimension. Here is a scree plot for the PCA transform used on the facial images.

In [ ]:
plt.figure(figsize=(9, 6))
plt.plot(pca.explained_variance_ratio_)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance')

Another way to look at it is to plot the sum of the variances as a function of component count.

In [ ]:
plt.figure(figsize=(9, 6))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance');

Based on these plots, if you had reduced the number of dimensions to 50 instead of 150, would you expect the restored facial images to look substantially different? If you're not sure, try it and see.

## Visualizing high-dimensional data

A practical use for PCA is visualizing high-dimensional data. Most machine-learning models are trained with datasets containing many dimensions. For example, a dataset of 100x100-pixel images contains 10,000 dimensions. Humans can only visualize data in two or at most three dimensions. Using PCA to reduce hundreds (or thousands) of dimensions to two or three and then plotting the reduced dataset helps us visualize clusters of data in high-dimensional datasets. We'll demonstrate by using PCA to reduce the digits used in the previous example to first two and then three dimensions and plotting the results.

Start by loading the dataset, reducing it to two dimensions, and plotting the result.

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
pca = PCA(n_components=2, random_state=0)
pca_digits = pca.fit_transform(digits.data)

plt.figure(figsize=(12, 8))
plt.scatter(pca_digits[:, 0], pca_digits[:, 1], c=digits.target, cmap=plt.cm.get_cmap('Paired', 10))
plt.colorbar(ticks=range(10))
plt.clim(-0.5, 9.5)

Reduced to two dimensions, the digits form somewhat distinct clusters, albeit with some overlap. 0s, represented by the light blue dots, seem to have little overlap with other digits, while 9s and 5s exhibit some similarity, at least when viewed in two dimensions. Let's try reducing the dataset to three dimensions rather than two to see if additional separation is evident with the extra dimension.

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D

digits = load_digits()
pca = PCA(n_components=3, random_state=0)
pca_digits = pca.fit_transform(digits.data)

ax = plt.figure(figsize=(12, 8)).add_subplot(111, projection='3d')
ax.scatter(xs = pca_digits[:, 0], ys = pca_digits[:, 1], zs = pca_digits[:, 2], c=digits.target, cmap=plt.cm.get_cmap('Paired', 10))

Based on this visualization, is it likely that a machine-learning model could be trained to distinguish between handwritten digits with a high degree of accuracy? Observe that if you rotate the plot around various axes, you can find a reasonable degree of separation between any of the clusters.

## Data visualization with t-SNE

PCA isn't the only way to reduce the dimensionality of data. Another method — one that is particulary well-suited for visualizing high-dimensional data — is [t-distributed stochastic neighbor embedding](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding), or t-SNE. Scikit-learn implements t-SNE in its [TSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) class. Let's use it to visualize the digits dataset in two dimensions and see how it compares to the PCA visualization.

In [ ]:
%matplotlib inline
from sklearn.manifold import TSNE

digits = load_digits()
tsne = TSNE(n_components=2, random_state=0)
tsne_digits = tsne.fit_transform(digits.data)

plt.figure(figsize=(12, 8))
plt.scatter(tsne_digits[:, 0], tsne_digits[:, 1], c=digits.target, cmap=plt.cm.get_cmap('Paired', 10))
plt.colorbar(ticks=range(10))
plt.clim(-0.5, 9.5)

You can see that t-SNE does a better job of separating groups of digits into clusters, indicating there are patterns in the data that machine learning can exploit. The chief drawback is that t-SNE is compute-intensive, which means it can take a prohibitively long time to run on large datasets. One way to mitigate that is to run t-SNE on a subset of rows rather than the entire dataset. Another strategy is to use PCA to reduce the number of dimensions, and then subject the PCAed dataset to t-SNE.

# PCA-based anomaly detection

Anomaly detection is an application of unsupervised learning that detects anomalous samples in datasets and data streams. Airbus uses it to [predict failures in jet engines](https://customers.microsoft.com/en-us/story/858578-airbus-defense-and-intelligence-azure) and [detect anomalies in telemetry data](https://blog.tensorflow.org/2020/04/how-airbus-detects-anomalies-iss-telemetry-data-tfx.html) beamed down from the International Space Station. Credit-card companies use it to detect fraudulent transactions. One of the more popular ways to build anomaly-detection models relies on [principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis), or PCA. PCA-based anomaly-detection models work by using PCA to transform a dataset, inverting the transform to attempt to restore the original dataset, and measuring the differences between the samples in the original dataset and the restored dataset. Anomalous samples are likely to exhibit a higher difference (loss) than "normal" samples, which enables future samples to be identified as normal or anomalous by computing the loss and comparing it to a chosen loss threshold.

![](Images/airplane.jpg)

This notebook applies PCA-based anomaly detection to a subset of a [dataset published by NASA ](https://www.kaggle.com/vinayak123tyagi/bearing-dataset) to predict failures in bearings. The dataset contains vibration data for four bearings supporting a rotating shaft with a radial load of 6,000 pounds applied to it. The bearings were run to failure, and vibration data was captured by PCB 353B33 high-sensitivity quartz accelerometers at regular intervals until failure occurred. This is analogous to the types of models aircraft manufacturers use to perform predictive maintenance on aircraft and aircraft engines. And it's a great example of how machine learning is used in industry to save lives and solve business problems.

## Load and visualize the data

The first step is to load the dataset, and then plot the data captured by the vibration sensors to get a picture of what happened during the test.

In [ ]:
import pandas as pd

df = pd.read_csv('Data/bearings.csv', index_col=0, parse_dates=[0])
df.head()

The dataset contains 984 samples. Each sample contains vibration data for four bearings, and the samples were taken 10 minutes apart. Plot the vibration data for all four bearings as a time series.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

df.plot(figsize = (12, 6))

About four days into the test, vibrations in bearing #1 began increasing. They spiked a day later, and about two days after that, bearing #1 suffered a catastrophic failure. Our goal is to build a model that recognizes increased vibration as a sign of impending failure, and PCA is the perfect tool for the job.

## Use PCA to transform and restore the data

The next step is to extract samples representing "normal" operation from the dataset, and reduce four dimensions to one using PCA. Then apply the same PCA transform to the remainder of the dataset, invert the transform, and plot the "restored" dataset to visualize the loss incurred when a PCA transform is applied and inverted. Start by reducing the dataset to one dimension with PCA and plotting the result.

In [ ]:
from sklearn.decomposition import PCA

x_train = df['2004-02-12 10:32:39':'2004-02-13 23:42:39']
x_test = df['2004-02-13 23:52:39':]

pca = PCA(n_components=1, random_state=0)
x_train_pca = pd.DataFrame(pca.fit_transform(x_train))
x_train_pca.index = x_train.index

x_test_pca = pd.DataFrame(pca.transform(x_test))
x_test_pca.index = x_test.index

df_pca = pd.concat([x_train_pca, x_test_pca])
df_pca.plot(figsize = (12, 6))
plt.legend().remove()

Now invert the PCA transform and plot the "restored" dataset.

In [ ]:
df_restored = pd.DataFrame(pca.inverse_transform(df_pca), index=df_pca.index)
df_restored.plot(figsize = (12, 6))

Examine the original datatset and the restored dataset in numerical form and compare the two.

In [ ]:
df.head()

In [ ]:
df_restored.head()

It is obvious that loss was incurred by applying and inverting the transform. The magnitude of the loss is roughly the same in all four columns, but that's not surprising given that the first five lines of each dataset represent normal operation. Let's define a function that computes the loss in a range of samples.

In [ ]:
import numpy as np

def get_anomaly_scores(df_original, df_restored):
    loss = np.sum((np.array(df_original) - np.array(df_restored)) ** 2, axis=1)
    loss = pd.Series(data=loss, index=df_original.index)
    return loss

Now apply that function to all of the samples in the original dataset and the restored dataset and plot the differences over time.

In [ ]:
scores = get_anomaly_scores(df, df_restored)
scores.plot(figsize = (12, 6))

It is apparent from this plot that the loss is very small when all four bearings are operating normally, but that the loss begins to rise when one or more bearings exhibits greater-than-normal vibration. It is also apparent that when the loss rises above a threshold value of approximately 0.002, that's an indication that a bearing might fail.

## Use PCA to detect anomalous behavior

Now that we've selected a tentative loss threshold, we can use it to detect anomalous behavior in the bearing set. Begin by defining a function that takes a sample as input and returns `True` or `False` indicating whether the sample is anomalous by applying and inverting a PCA transform, measuring the loss, and comparing it to a specified loss threshold.

In [ ]:
def is_anomaly(data, pca, threshold):
    pca_data = pca.transform(data)
    restored_data = pca.inverse_transform(pca_data)
    loss = np.sum((data - restored_data) ** 2)
    return loss > threshold

Apply the function to a row early in the time series that represents normal behavior.

In [ ]:
x = [df.loc['2004-02-16 22:52:39']]
is_anomaly(x, pca, 0.002)

Apply the function to a row later in the time series that represents anomalous behavior.

In [ ]:
x = [df.loc['2004-02-18 22:52:39']]
is_anomaly(x, pca, 0.002)

Apply the function to all the samples in the dataset and shade anomalous samples red in order to visualize when anomalous behavior is detected.

In [ ]:
df.plot(figsize = (12, 6))

for index, row in df.iterrows():
    if is_anomaly([row], pca, 0.002):
        plt.axvline(row.name, color='r', alpha=0.2)

Repeat this procedure, but this time use a loss threshold of 0.0002 rather than 0.002 to detect anomalous behavior.

In [ ]:
df.plot(figsize = (12, 6))

for index, row in df.iterrows():
    if is_anomaly([row], pca, 0.0002):
        plt.axvline(row.name, color='r', alpha=0.2)

You can adjust the sensitivity of the model by adjusting the threshold value used to identify anomalies. Using a loss threshold of 0.002 predicts bearing failure about two days before it occurs, while a loss threshold of 0.0002 predicts the failure about three days before. You typically want to choose a loss threshold that predicts failure as early as possible without raising false alarms.

## PCA for noise reduction

One practical use for PCA is filtering noise from data. The basic approach is to PCA-transform the dataset, and then perform an inverse transform, taking the dataset from *m* dimensions to *n* and then back to *m*. Here we demonstrate by artificially adding noise to the handwritten-digits dataset included with Scikit-learn. That dataset contains 1,797 images, each an 8x8 array of numbers comprising 64 dimensions. Start by loading the dataset and showing the first 50 images.

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()
fig, axes = plt.subplots(5, 10, figsize=(12, 7), subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(digits.images[i], cmap=plt.cm.gray_r)

Introduce random noise to the dataset and plot the results.

In [ ]:
np.random.seed(42)
noisy = np.random.normal(digits.data, 4)
fig, axes = plt.subplots(5, 10, figsize=(12, 7), subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(noisy[i].reshape(8, 8), cmap=plt.cm.gray_r)

Use PCA to reduce the number of dimensions. Rather than specify the number of dimensions (components), we'll specify that we want to reduce the amount of information in the dataset to 50%. We'll let Scikit decide how many dimensions will remain, and then show the number.

In [ ]:
pca = PCA(0.5, random_state=0).fit(noisy)
pca.n_components_

PCA reduced the number of dimensions from 64 to 12, but the 12 remaining dimensions contain 50% of the information in the original 64. Now reconstruct the dataset from the reduced version.

In [ ]:
pca_digits = pca.transform(noisy)
unpca_digits = pca.inverse_transform(pca_digits)

fig, axes = plt.subplots(5, 10, figsize=(12, 7), subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(unpca_digits[i].reshape(8, 8), cmap=plt.cm.gray_r)

The reconstructed dataset isn't quite as clean as the original, but it's clean enough that you can make out most of the numbers.